<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%85%D8%A7%D9%83%D8%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# استخدام الدالة
data = calculate_macd(data)


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import sqlite3

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

# إنشاء اتصال بقاعدة بيانات SQLite
conn = sqlite3.connect('gold_prices.db')

# إنشاء جدول لتخزين بيانات الأسعار
def create_table():
    with conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS gold_prices (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                time TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER
            )
        ''')
create_table()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def insert_data(df):
    """Insert data into SQLite database."""
    with conn:
        df.to_sql('gold_prices', conn, if_exists='append', index=False)

# جلب البيانات وإدخالها إلى قاعدة البيانات
df_gold_prices = get_and_clean_gold_prices()
if df_gold_prices is not None:
    insert_data(df_gold_prices)

def fetch_data():
    """Fetch data from SQLite database."""
    query = "SELECT * FROM gold_prices"
    return pd.read_sql_query(query, conn)

# استخدام الدالة لاسترجاع البيانات
data = fetch_data()
print(data)

# إغلاق الاتصال بقاعدة البيانات وMT5
conn.close()
mt5.shutdown()


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import sqlite3

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

# إنشاء اتصال بقاعدة بيانات SQLite
conn = sqlite3.connect('gold_prices.db')

# إنشاء جدول لتخزين بيانات الأسعار
def create_table():
    with conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS gold_prices (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                time TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER,
                target REAL
            )
        ''')
create_table()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]

        # إضافة العمود 'target'
        df['target'] = df['close'].shift(-1)  # الهدف هو السعر في اليوم التالي

        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def insert_data(df):
    """Insert data into SQLite database."""
    with conn:
        df.to_sql('gold_prices', conn, if_exists='append', index=True)

# جلب البيانات وإدخالها إلى قاعدة البيانات
df_gold_prices = get_and_clean_gold_prices()
if df_gold_prices is not None:
    insert_data(df_gold_prices)

def fetch_data():
    """Fetch data from SQLite database."""
    query = "SELECT * FROM gold_prices"
    return pd.read_sql_query(query, conn)

# استخدام الدالة لاسترجاع البيانات
data = fetch_data()
print(data)

# إغلاق الاتصال بقاعدة البيانات وMT5
conn.close()
mt5.shutdown()


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import sqlite3

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

# إنشاء اتصال بقاعدة بيانات SQLite
conn = sqlite3.connect('gold_prices.db')

def reset_table():
    """Delete the existing table if necessary."""
    with conn:
        conn.execute('DROP TABLE IF EXISTS gold_prices')

def create_table():
    with conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS gold_prices (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                time TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER,
                target REAL
            )
        ''')
reset_table()  # حذف الجدول القديم
create_table()  # إنشاء الجدول الجديد

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]

        # إضافة العمود 'target'
        df['target'] = df['close'].shift(-1)  # الهدف هو السعر في اليوم التالي

        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def insert_data(df):
    """Insert data into SQLite database."""
    with conn:
        df.to_sql('gold_prices', conn, if_exists='append', index=True)

# جلب البيانات وإدخالها إلى قاعدة البيانات
df_gold_prices = get_and_clean_gold_prices()
if df_gold_prices is not None:
    insert_data(df_gold_prices)

def fetch_data():
    """Fetch data from SQLite database."""
    query = "SELECT * FROM gold_prices"
    return pd.read_sql_query(query, conn)

# استخدام الدالة لاسترجاع البيانات
data = fetch_data()
print(data)

# إغلاق الاتصال بقاعدة البيانات وMT5
conn.close()
mt5.shutdown()


In [ ]:
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        # تحويل الأعمدة إلى أنواع مناسبة
        data['close'] = pd.to_numeric(data['close'], errors='coerce')
        data['open'] = pd.to_numeric(data['open'], errors='coerce')
        data['high'] = pd.to_numeric(data['high'], errors='coerce')
        data['low'] = pd.to_numeric(data['low'], errors='coerce')

        # تعبئة القيم الفارغة
        data.fillna(0, inplace=True)

        # إضافة الميزات
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data


In [ ]:
def build_strategy(data):
    try:
        data.fillna(0, inplace=True)
        data = data.infer_objects()  # استنتاج الأنواع بعد ملء القيم الفارغة

        features = data.drop(['target', 'time'], axis=1)  # استبعاد عمود 'time'
        target = data['target']

        # تأكد من أن جميع الأعمدة في features هي int أو float
        if not all(features.dtypes.isin([np.int64, np.float64])):
            print("Invalid data types in features.")
            return

        # هنا يمكنك متابعة بناء النموذج، مثل استخدام XGBoost
        # ...
    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")
